In [1]:
import time
from functools import partial
import itertools
from nltk import ngrams
from nltk.corpus import stopwords

In [2]:
import sys
sys.path.append("../module/")
import menconn
from typelinking import *

settings = load_settings('..', 'ja')

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


INFO:tensorflow:Restoring parameters from ../en_model/model.ckpt


In [5]:
def parse_tokenize(sentence, mtag):
    sentence_parsed = mtag.parse(sentence)
    return sentence_parsed.split()

def get_japanese_stopwords():
    stps = stopwords.words('japanese')
    return stps + [''.join(list(stp)) for stp in list(itertools.product(stps, stps))]

def get_candidate_mentions(sentence, tokenize, trie, stps=[], sep=' '):
    sentence_parsed = tokenize(sentence)
    ts = []
    for n in range(1, 10):
        n_grams = ngrams(sentence_parsed, n)
        for grams in n_grams:
            ts.append(sep.join(list(grams)))
    return [t for t in ts if trie.get(t) is not None and t not in stps]

In [6]:
import MeCab
trie = settings[3]
mtag = MeCab.Tagger('-Owakati')
sentence = """
バラク・オバマは基本的に言ってインテリ層に人気のある黒人だったが、
ドナルド・トランプは白人主義者や陰謀論者から人気を集めている。
"""
stps = get_japanese_stopwords()
tokenize = partial(parse_tokenize, mtag=mtag)

start = time.time()
ts = get_candidate_mentions(sentence, tokenize, trie, stps, sep='')
end = time.time()
print('Time:{}'.format(end-start))
print(ts)

Time:0.00500035285949707


In [9]:
tokenize = partial(menconn.ja_tokenize, ts=ts)

start = time.time()
results = analyze(sentence, ts, tokenize, *settings, lang='ja')
end = time.time()
print("Time:{}".format(end-start))

Time:0.022411108016967773


In [8]:
results

[{'mention': 'バラク', 'pred': 'バラク (チャガタイ家)'},
 {'mention': 'オバマ', 'pred': 'バラク・オバマ'},
 {'mention': '基本', 'pred': '基本'},
 {'mention': '的', 'pred': '的 (弓道)'},
 {'mention': 'インテリ', 'pred': 'インテリ'},
 {'mention': '層', 'pred': '層 (数学)'},
 {'mention': '人気', 'pred': '人気'},
 {'mention': '黒人', 'pred': '黒人'},
 {'mention': 'ドナルド', 'pred': 'ドナルドダック'},
 {'mention': 'トランプ', 'pred': 'トランプ'},
 {'mention': '白人', 'pred': '白人'},
 {'mention': '主義', 'pred': '主義'},
 {'mention': '者', 'pred': '個人'},
 {'mention': '陰謀', 'pred': '陰謀'},
 {'mention': '論', 'pred': '論'},
 {'mention': '者', 'pred': '個人'},
 {'mention': '人気', 'pred': '人気'},
 {'mention': '集め', 'pred': 'コレクション'},
 {'mention': '主義者', 'pred': '主義者'},
 {'mention': '陰謀論', 'pred': '陰謀論'},
 {'mention': 'バラク・オバマ', 'pred': 'バラク・オバマ'},
 {'mention': 'ドナルド・トランプ', 'pred': 'ドナルド・トランプ'},
 {'mention': '陰謀論者', 'pred': '陰謀論'}]